<a href="https://colab.research.google.com/github/fjtm/Data/blob/master/MAC_KNN_first_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template for the homework


## Task
- Classify each document in one of 20 categories.
- The objective is obtain the better accuracy in the test set. You can use any library and model explained in the course.
- The delivery are a unique jupiter notebook with all the code. Must run in the course Anaconda environment. Not use additional libraries.
- Send the notebook named homework\_[name]\_[surename].ipynb to sueiras@gmail.com before November 20th.

## Template structure

- A Jupiter notebook template is provided to do the task. Structure:
  - Read the train and validation data.
  - Transform to generate numerical features.: Build your transformations here
  - Model: Build your model or models here. Check the accuracy over the validation set.
  - Evaluate results: Build your scoring function here and apply it over the test set.
- You need to complete the transform and model steps to achieve the best result in the evaluation metric, the accuracy, in test set.
- Is completely forbidden load and use the test set except once in the final evaluate results step.

## Evaluation

- Exercise evaluated in 0-10 range points.
- To obtain 5 points you must deliver a notebook without errors that provide a solution whit a minimum accuracy of 67%.
- If you obtain an accuracy over 87% you have 10 points.
- Intermediated accuracies between 67% and 87% obtain intermediated points proportionally, but depending of the quality of the work is possible to reduce or increase a maximum of 2 the points assigned automatically by accuracy. 
 

In [0]:
# Header
from __future__ import print_function

import pandas as pd
import numpy as np


## 01 Load data

In [2]:
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train', remove=('footers'), shuffle=True, random_state=23)
                 #remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)

print(twenty_train.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [0]:
index = [i[0] for i in sorted(enumerate(twenty_train.target), key=lambda x:x[1])]
twenty_train.target = twenty_train.target[index]
twenty_train.data = [twenty_train.data[ind] for ind in index]

In [0]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
encoding_fit = enc.fit(twenty_train.target.reshape(-1, 1))

model_ohe = encoding_fit.fit(twenty_train.target.reshape(-1, 1))
twenty_trn = model_ohe.transform(twenty_train.target.reshape(-1, 1)).toarray()

In [5]:
twenty_trn

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [6]:
# Separate train and validation
from sklearn.model_selection import train_test_split

# Recommended 20% to validation. 
text_trn, text_val, Y_train, Y_val = train_test_split(twenty_train.data, twenty_trn, test_size=0.2, random_state = 23, shuffle = True)
print(len(text_trn), len(text_val))

9051 2263


In [7]:
Y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0.])

## 02 Text encoding

Tokenizamos las palabras

In [8]:
print("\n".join(text_trn[10].split("\n")))

From: boyle@cactus.org (Craig Boyle)
Subject: Re: New break pads & exhausts after 96K km (60K mi) on '90 Maxima?
Organization: Capital Area Central Texas UNIX Society, Austin, Tx
Lines: 78

In article <1993Apr16.000601.14223@jarvis.csri.toronto.edu> rkim@eecg.toronto.edu (Ryan Kim) writes:
>
>Hi, maybe someone can help me here...
>I am looking to buy this 1990 Nissan Maxima GXE for CDN$14000 right now.

So its an automatic? Don't know if US spec=CDN spec. for Maximas.

>The car has 96000 km (or about 60000 miles) on it.
>A typical mileage for 1990 cars seem to be about 70000 km (or about 43K mi).
>The seller just informed me that when he brought the car in for certification
>he was told that the front break pads and the exhausts had to be replaced
>to meet the legal standards.  (He said he will replace the components before
>selling the car to me.)
>
>Being copmletely ignorant to the technical stuff on cars, I don't know
>what this could mean...
>Is 96K km about the time typical for re

In [9]:
text_trn[0]

'From: kennejs@a.cs.okstate.edu (KENNEDY JAMES SCOT)\nSubject: Drug Use Up Among U.S. Eighth-graders\nOrganization: Oklahoma State University, Computer Science, Stillwater\nKeywords: youths drugs LSD inhalants\nLines: 87\n\n\nThe article that follows was taken from the Wednesday, April 14, 1993\nissue of USA Today ("Drug Use Up Among U.S. eighth-graders" by Mike\nSnider, p. 6D).\n\n    A new national survey says drugs are easier to get, more teens are\n    using them and fewer deem drug use as risky. \n\n    For the last two years, government officials have trumpeted results\n    from the National High School Survey as signs that the drug war is\n    being won.  But this year, officials are retreating - drug use by\n    eighth-graders has risen, according to the survey of 50,000 students\n    nationwide.\n\n    Possible reason for the increase:  more experimentation.  Why?  If\n    drug use dropped during the \'80s, eventually some students will\n    have fewer "drug-using contemporari

In [0]:
def clean_head_body(data):  
  text_complete = []
  for j, text in enumerate(data):
    head = []
    sentence_sinal = []
    for i, sentence in enumerate(text.split("\n")):
      if sentence != "":
        if "subject" in sentence.lower():
          head.append(sentence[len("subject")::])
        if "summary" in sentence.lower():
          head.append(sentence[len("summary")::])
        if "keywords" in sentence.lower():
          head.append(sentence[len("keywords")::])
        if "organization" in sentence.lower():
          head.append(sentence[len("organization")::])
      else:
        break
    body = text.split("\n")[i+1::]
    text_final = head + body
    text_complete.append((' ').join(text_final))
  return(text_complete)

In [0]:
text_trn = clean_head_body(text_trn)
text_val = clean_head_body(text_val) 

In [0]:
import re

def remove_non_alphanumeric(text):
  text_clean = []
  for sentence in text:
    text_clean.append(re.compile('\W').sub(" ",sentence))
                      
  return(text_clean)
                      
                      
text_trn = remove_non_alphanumeric(text_trn)
text_val = remove_non_alphanumeric(text_val)

In [13]:
text_trn[10]

'  Re  New break pads   exhausts after 96K km  60K mi  on  90 Maxima    Capital Area Central Texas UNIX Society  Austin  Tx In article  1993Apr16 000601 14223 jarvis csri toronto edu  rkim eecg toronto edu  Ryan Kim  writes     Hi  maybe someone can help me here     I am looking to buy this 1990 Nissan Maxima GXE for CDN 14000 right now   So its an automatic  Don t know if US spec CDN spec  for Maximas    The car has 96000 km  or about 60000 miles  on it   A typical mileage for 1990 cars seem to be about 70000 km  or about 43K mi    The seller just informed me that when he brought the car in for certification  he was told that the front break pads and the exhausts had to be replaced  to meet the legal standards    He said he will replace the components before  selling the car to me      Being copmletely ignorant to the technical stuff on cars  I don t know  what this could mean     Is 96K km about the time typical for replacing the above mentioned items   Or is this an indication that 

In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
  text_clean = []
  for sentence in text:
    text_clean.append(' '.join(word for word in sentence.lower().split() if word not in STOPWORDS))
  return(text_clean)

text_trn = remove_stopwords(text_trn)
text_val = remove_stopwords(text_val)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
text_trn[10]

'new break pads exhausts 96k km 60k mi 90 maxima capital area central texas unix society austin tx article 1993apr16 000601 14223 jarvis csri toronto edu rkim eecg toronto edu ryan kim writes hi maybe someone help looking buy 1990 nissan maxima gxe cdn 14000 right automatic know us spec cdn spec maximas car 96000 km 60000 miles typical mileage 1990 cars seem 70000 km 43k mi seller informed brought car certification told front break pads exhausts replaced meet legal standards said replace components selling car copmletely ignorant technical stuff cars know could mean 96k km time typical replacing mentioned items indication car abused first set brake pads front fine car eats set every 15k miles fact replacing muffler also ok would things break replaced soon mileage fairly low typical fwd stuff cv joints check maintenance records manufacturers requirements valve adjustments timing belt changes 60k mile service often expensive make sure done everything seller told used car highway lot know

In [0]:
def remove_one_word(text):
  text_clean = []
  for sentence in text:
    text_clean.append(' '.join(word for word in sentence.split() if len(word) > 1))
  return(text_clean)  

In [0]:
text_trn = remove_one_word(text_trn)
text_val = remove_one_word(text_val)

In [18]:
text_trn[10]

'new break pads exhausts 96k km 60k mi 90 maxima capital area central texas unix society austin tx article 1993apr16 000601 14223 jarvis csri toronto edu rkim eecg toronto edu ryan kim writes hi maybe someone help looking buy 1990 nissan maxima gxe cdn 14000 right automatic know us spec cdn spec maximas car 96000 km 60000 miles typical mileage 1990 cars seem 70000 km 43k mi seller informed brought car certification told front break pads exhausts replaced meet legal standards said replace components selling car copmletely ignorant technical stuff cars know could mean 96k km time typical replacing mentioned items indication car abused first set brake pads front fine car eats set every 15k miles fact replacing muffler also ok would things break replaced soon mileage fairly low typical fwd stuff cv joints check maintenance records manufacturers requirements valve adjustments timing belt changes 60k mile service often expensive make sure done everything seller told used car highway lot know

In [19]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
def lemmatizer(text):
  text_new = []
  for sentence in text:
    sentence_new = []
    for word in sentence.split():
      sentence_new.append(WordNetLemmatizer().lemmatize(word,'v'))
    text_new.append((' ').join(sentence_new))
  return(text_new)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
text_trn = lemmatizer(text_trn)
text_val = lemmatizer(text_val)

In [21]:
text_trn[10]

'new break pad exhaust 96k km 60k mi 90 maxima capital area central texas unix society austin tx article 1993apr16 000601 14223 jarvis csri toronto edu rkim eecg toronto edu ryan kim write hi maybe someone help look buy 1990 nissan maxima gxe cdn 14000 right automatic know us spec cdn spec maximas car 96000 km 60000 miles typical mileage 1990 cars seem 70000 km 43k mi seller inform bring car certification tell front break pad exhaust replace meet legal standards say replace components sell car copmletely ignorant technical stuff cars know could mean 96k km time typical replace mention items indication car abuse first set brake pad front fine car eat set every 15k miles fact replace muffler also ok would things break replace soon mileage fairly low typical fwd stuff cv joint check maintenance record manufacturers requirements valve adjustments time belt change 60k mile service often expensive make sure do everything seller tell use car highway lot know verify see paint chip away tiny do

In [0]:
#from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
#porter = PorterStemmer()
stemmer = SnowballStemmer("english")
def normalize_words(text):
  text_new = []
  for sentence in text:
    sentence_new = []
    for word in sentence.split():
      #sentence_new.append(porter.stem(word))
      sentence_new.append(stemmer.stem(word))
    text_new.append((' ').join(sentence_new))
  return(text_new) 

In [0]:
text_trn = normalize_words(text_trn)
text_val = normalize_words(text_val)

In [24]:
text_trn[10]

'new break pad exhaust 96k km 60k mi 90 maxima capit area central texa unix societi austin tx articl 1993apr16 000601 14223 jarvi csri toronto edu rkim eecg toronto edu ryan kim write hi mayb someon help look buy 1990 nissan maxima gxe cdn 14000 right automat know us spec cdn spec maxima car 96000 km 60000 mile typic mileag 1990 car seem 70000 km 43k mi seller inform bring car certif tell front break pad exhaust replac meet legal standard say replac compon sell car copmlet ignor technic stuff car know could mean 96k km time typic replac mention item indic car abus first set brake pad front fine car eat set everi 15k mile fact replac muffler also ok would thing break replac soon mileag fair low typic fwd stuff cv joint check mainten record manufactur requir valv adjust time belt chang 60k mile servic often expens make sure do everyth seller tell use car highway lot know verifi see paint chip away tini dot front edg hood though well one common cite method identifi car highway mile might 

In [25]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [26]:
# The maximum number of words to be used. (most frequent)
max_features = 8000
# Max number of words in each complaint.
maxlen = 300

tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(text_trn)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 75026 unique tokens.


In [27]:
import operator

sorted_x = sorted(tokenizer.word_counts.items(), key=operator.itemgetter(1), reverse = True)
sorted_x

[('write', 7947),
 ('edu', 7781),
 ('use', 7193),
 ('one', 7157),
 ('would', 6621),
 ('get', 6028),
 ('articl', 5510),
 ('say', 5442),
 ('know', 4986),
 ('ax', 4985),
 ('like', 4882),
 ('peopl', 4509),
 ('make', 4496),
 ('think', 4490),
 ('com', 4488),
 ('go', 4326),
 ('univers', 4256),
 ('time', 3944),
 ('also', 3262),
 ('see', 3225),
 ('year', 3037),
 ('system', 3012),
 ('work', 2945),
 ('new', 2877),
 ('good', 2850),
 ('take', 2816),
 ('want', 2691),
 ('right', 2655),
 ('could', 2636),
 ('need', 2613),
 ('well', 2610),
 ('problem', 2585),
 ('even', 2515),
 ('come', 2510),
 ('look', 2463),
 ('way', 2453),
 ('thing', 2440),
 ('window', 2437),
 ('give', 2432),
 ('may', 2414),
 ('find', 2393),
 ('file', 2369),
 ('state', 2367),
 ('god', 2334),
 ('two', 2330),
 ('us', 2311),
 ('mani', 2249),
 ('first', 2217),
 ('question', 2204),
 ('much', 2180),
 ('tri', 2158),
 ('call', 2060),
 ('run', 2047),
 ('point', 2004),
 ('comput', 1994),
 ('post', 1982),
 ('program', 1961),
 ('mean', 1908),
 ('

In [0]:
X_train = tokenizer.texts_to_sequences(text_trn)
X_valid = tokenizer.texts_to_sequences(text_val)

In [0]:
import tensorflow as tf

X_trn = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen, padding='pre', truncating='pre')
X_val = tf.keras.preprocessing.sequence.pad_sequences(X_valid, maxlen=maxlen, padding='pre', truncating='pre')

## 03 Model and score function

In [0]:
from tensorflow.keras.callbacks import EarlyStopping

In [31]:
# Model
dim_embedings = 200 #Dimension of the embedings vector
num_hidden_rnn = 128 #Num of neurons in the Recurent network 

#from keras import layers, models, optimizers

print('Build model 1 - Basic model...')

# LAYER 1: inputs
seq_prev_input = tf.keras.layers.Input(shape=(maxlen,), dtype='int32') 

# LAYER 2: Create embedings
embeds = tf.keras.layers.Embedding(max_features, dim_embedings)(seq_prev_input)

spaci_drop = tf.keras.layers.SpatialDropout1D(0.75)(embeds)

# LAYERS 3: RNN - forwards LSTM with dropout
droput_percent = 0.7

forwards2   = tf.keras.layers.LSTM(num_hidden_rnn, return_sequences=False,
                 dropout=droput_percent, recurrent_dropout=droput_percent, name='Forward_2')(spaci_drop)

# LAYER 4: Dense layer to outputs - softmax activation
output = tf.keras.layers.Dense(20, activation='softmax')(forwards2)

# Model Architecture defined
model_1 = tf.keras.models.Model(inputs=seq_prev_input, outputs=output)
model_1.summary()

Build model 1 - Basic model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 200)          1600000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 300, 200)          0         
_________________________________________________________________
Forward_2 (LSTM)             (None, 128)               168448    
_________________________________________________________________
dense (Dense)                (None, 20)                2580      
Total params: 1,771,028
Trainable params: 1,771,028
Non-

In [0]:
rms_optimizer = tf.keras.optimizers.RMSprop(lr=0.001)
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
batch_size = 256

print("Train...")
#tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./tensorboard/sentiment/simpleLSTM', histogram_freq=1)
history = model_1.fit(X_trn, Y_train, batch_size=batch_size, epochs=200, validation_data=(X_val, Y_val),
                     callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10, restore_best_weights=True)])

Train...
Train on 9051 samples, validate on 2263 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
2816/9051 [========>.....................] - ETA: 21s - loss: 2.9957 - acc: 0.0533

KeyboardInterrupt: ignored

## 04 Evaluate valid data

In [0]:
score = model_1.evaluate(X_val, Y_val)
print("Test loss %.4f" % score[0])
print("Test accuracy %.4f" % score[1])

In [0]:
# Confussion matrix
#from sklearn.metrics import accuracy_score, confusion_matrix


#Calculate accuracy with sklearn
#print('Accuracy valid: ', model_1.evaluate(X_validate, Y_validate))
#pd.DataFrame(confusion_matrix(y_val, pred_val))


## 05 Evaluate test data
- Don't edit after this!!!
- Execute only ONCE whit the optimal model selected based on the validation accuracy metric calculated over multiple experiments.

In [0]:
twenty_test = fetch_20newsgroups(subset='test', remove = 'footers',
                                 shuffle=True, random_state=23)

text_test = clean_head_body(twenty_test.data)
text_test = remove_non_alphanumeric(text_test)
text_test = remove_stopwords(text_test)
text_test = remove_one_word(text_test)
text_test = lemmatizer(text_test)
text_test = normalize_words(text_test)
text_test = tokenizer.texts_to_sequences(text_test)
X_test = tf.keras.preprocessing.sequence.pad_sequences(text_test, maxlen=maxlen, padding='pre', truncating='pre')
Y_test = model_ohe.transform(twenty_test.target.reshape(-1, 1)).toarray()

score = model_1.evaluate(X_test, Y_test)
print("Test loss %.4f" % score[0])
print("Test accuracy %.4f" % score[1])

In [0]:
X_test.shape